In [1]:
import sys
sys.path.append('../code')

from settings import VECTOR_SIZE,  CLASSIFIER_EPOCHS


In [2]:
from gensim.models import Word2Vec
word_vectors = Word2Vec.load('walks/model').wv

In [3]:
from pathlib import Path
import pandas as pd
from models import ClassifierSimple
import torch
model = ClassifierSimple()    

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if Path('rdf2vecClassfier.pth').is_file():
    print('found trained model! Loading :)')
    model.load_state_dict(torch.load('rdf2vecClassfier.pth'))
    history = pd.read_csv('log.csv')
    model = model.to(device)
else:
    print('model not found. Train it with ''train_rdf2vec_classifier.ipynb')

2022-11-17 15:16:39.136467: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-17 15:16:39.136488: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


found trained model! Loading :)


In [4]:
from rdflib import Graph, URIRef
from rdflib.extras.external_graph_libs import rdflib_to_networkx_multidigraph
from utils_graph import clean_graph, get_entities


g_train = Graph()
g_val = Graph()
g_test = Graph()

g_train = g_train.parse('FB15k-237/train_1000.nt', format='nt')
g_val   = g_val.parse('FB15k-237/valid_1000.nt', format='nt')
g_test  = g_test.parse('FB15k-237/test_1000.nt', format='nt')


# clean graphs 
# number of triples removed should be low, a few hundred
print(f"removed {clean_graph(g_train,word_vectors)} triples from training set")
print(f"removed {clean_graph(g_val,word_vectors)} triples from validation set")
print(f"removed {clean_graph(g_test,word_vectors)} triples from test set")

entities = get_entities((g_train,g_val,g_test))

removed 2 triples from training set
removed 2 triples from validation set
removed 0 triples from test set


In [5]:
from utils_eval import eval_ranks, mean_rank, mean_reciprocal_rank, hitsAt

In [7]:
# tested scoring on cpu
import gc
torch.set_num_threads(32)
with torch.no_grad():
    head_ranks, tail_ranks,pt = eval_ranks(model,g_train,lambda x: word_vectors[x],force_cpu=True,filtered=True,bit16tensors=False)
    
    print(f"MR: {mean_rank(head_ranks, tail_ranks)}")
    print(f"MRR: {mean_reciprocal_rank(head_ranks, tail_ranks)}")
    
    print(f"H@1: {hitsAt(head_ranks, tail_ranks,1)}")
    print(f"H@3: {hitsAt(head_ranks, tail_ranks,3)}")
    print(f"H@10: {hitsAt(head_ranks, tail_ranks,10)}")
    
    
    print(f"MR_head: {torch.mean(head_ranks)}")
    print(f"MRR_head: {torch.mean(1/head_ranks)}")
    
    print(f"H@1_head: {(head_ranks <= 1).sum()/len(tail_ranks)}")
    print(f"H@3_head: {(head_ranks <= 3).sum()/len(tail_ranks)}")
    print(f"H@10_head: {(head_ranks <= 10).sum()/len(tail_ranks)}")
    
    
    print(f"MR_tail: {torch.mean(tail_ranks)}")
    print(f"MRR_tail: {torch.mean(1/tail_ranks)}")
    
    print(f"H@1_tail: {(tail_ranks <= 1).sum()/len(tail_ranks)}")
    print(f"H@3_tail: {(tail_ranks <= 3).sum()/len(tail_ranks)}")
    print(f"H@10_tail: {(tail_ranks <= 10).sum()/len(tail_ranks)}")
    
    
gc.collect()
pt.stats_sum()


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 156/156 [02:57<00:00,  1.14s/it]

MR: 62.0576171875
MRR: 0.2724679112434387
H@1: 0.08667334914207458
H@3: 0.17284569144248962
H@10: 0.33717435598373413
MR_head: 49.41482925415039
MRR_head: 0.35735464096069336
H@1_head: 0.12324649095535278
H@3_head: 0.22545090317726135
H@10_head: 0.4008015990257263
MR_tail: 74.70040130615234
MRR_tail: 0.18758118152618408
H@1_tail: 0.05010019987821579
H@3_tail: 0.1202404797077179
H@10_tail: 0.27354708313941956


{'preprocessing': 4.458399113998894,
 'subgraph': 0.15552735196979484,
 'collect_embeddings': 0.006837044002168113,
 'copy embeddings into array': 16.40838050302409,
 'score_embeddings': 160.23794436401295,
 'rank_embeddings': 0.3474007089898805}

In [ ]:
print('here!')